In [18]:
import ee
ee.Authenticate(auth_mode='notebook')
ee.Initialize(project='global-booster-421311')


In [20]:
# Your catchments asset (update this if your asset path changes)
catchments = ee.FeatureCollection("projects/global-booster-421311/assets/LakeCatchments_NA")

landcover_codes = [0, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 170, 180]
weeks = list(range(1, 54))  # 1 to 53

# For a single year example (2019)
image = ee.ImageCollection('ESA/CCI/FireCCI/5_1').filterDate('2019-01-01', '2020-01-01').first()
year = 2019

def features_for_week(week):
    week = ee.Number(week)
    start_day = week.subtract(1).multiply(7).add(1)
    end_day = week.multiply(7).min(366)
    burned = image.select('BurnDate').gte(start_day).And(image.select('BurnDate').lte(end_day))
    landcover = image.select('LandCover')
    bands = [burned.And(landcover.eq(lc)).rename(f'burned_pix_lc_{lc}') for lc in landcover_codes]
    stack = ee.Image.cat(bands)
    stats = stack.reduceRegions(
        collection=catchments,
        reducer=ee.Reducer.sum().forEachBand(stack),
        scale=250
    ).map(lambda f: f.set({'year': year, 'week': week}))
    return stats

# Map over all weeks
weekly_collections = ee.List(weeks).map(lambda w: features_for_week(ee.Number(w)))
all_features = ee.FeatureCollection(weekly_collections).flatten()

# Remove geometry before export
def drop_geometry(feature):
    return feature.setGeometry(None)
all_features_no_geom = all_features.map(drop_geometry)

export_fields = ['Lake_ID', 'year', 'week'] + [f'burned_pix_lc_{lc}' for lc in landcover_codes]

task = ee.batch.Export.table.toDrive(
    collection=all_features_no_geom.select(export_fields),
    description='FireCCI51_BurnedPixels_Catchments_Weekly',
    fileFormat='CSV'
)
task.start()
print("Export started! Monitor in GEE Tasks or Google Drive.")


Export started! Monitor in GEE Tasks or Google Drive.


In [ ]:
import pandas as pd
from datetime import datetime, timedelta

def week_start_end(year, week):
    start = datetime(year, 1, 1) + timedelta(days=(week - 1) * 7)
    end = min(start + timedelta(days=6), datetime(year, 12, 31))
    return start.strftime('%Y/%m/%d'), end.strftime('%Y/%m/%d')

landcover_code_to_name = {
    0: "Cropland_rainfed",
    20: "Cropland_irrigated",
    30: "Mosaic_cropland",
    40: "Mosaic_natural_veg",
    50: "Tree_broad_evergreen",
    60: "Tree_broad_deciduous",
    70: "Tree_needle_evergreen",
    80: "Tree_needle_deciduous",
    90: "Tree_mixed",
    100: "Mosaic_tree_shrub",
    110: "Mosaic_herbaceous",
    120: "Shrubland",
    130: "Grassland",
    140: "Lichens_mosses",
    150: "Sparse_vegetation",
    170: "Tree_flooded_saline",
    180: "Shrub_herb_flooded"
}

# Load your exported CSV
df = pd.read_csv("extracted_gee/FireCCI51_BurnedPixels_Catchments_Weekly.csv")

# Drop system:index and .geo if present
to_drop = ['system:index', '.geo']
df = df.drop(columns=[c for c in to_drop if c in df.columns])

# Add week_start and week_end columns
df[['week_start', 'week_end']] = df.apply(
    lambda row: week_start_end(int(row['year']), int(row['week'])), axis=1, result_type='expand'
)

# Rename land cover columns to descriptive names
rename_dict = {
    f"burned_pix_lc_{code}": f"{name}"
    for code, name in landcover_code_to_name.items()
}
df = df.rename(columns=rename_dict)

# Desired column order
main_cols = ['Lake_ID', 'year', 'week', 'week_start', 'week_end']
other_cols = [col for col in df.columns if col not in main_cols]
df = df[main_cols + other_cols]

# Sort by Lake_ID, year, week
df = df.sort_values(by=['Lake_ID', 'year', 'week'])

# Save final table
df.to_csv('extracted_gee/FireCCI51_BurnedPixels_Catchments_Weekly.csv', index=False)

print("✅ Weekly summary with dates and descriptive burned pixel counts ready!")


✅ Weekly summary with dates and descriptive burned pixel counts ready!
